<a href="https://colab.research.google.com/github/ciepielajan/SDA_SMOG_Wawelski/blob/main/Jan/SDA_2021_04_17_aplikacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`wprowadzenie danych przez użytkownika`

In [22]:
data = "2017-12-03"
time = "11"
user_latitude = "50.057747"
user_longitude = "19.961374"

#Aplikacja

1 transformacji i modelu







In [1]:
!wget "https://github.com/ciepielajan/SDA_SMOG_Wawelski/raw/main/Jan/model/pipeline.pickle"

--2021-04-17 22:07:21--  https://github.com/ciepielajan/SDA_SMOG_Wawelski/raw/main/Jan/model/pipeline.pickle
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ciepielajan/SDA_SMOG_Wawelski/main/Jan/model/pipeline.pickle [following]
--2021-04-17 22:07:21--  https://raw.githubusercontent.com/ciepielajan/SDA_SMOG_Wawelski/main/Jan/model/pipeline.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2324280 (2.2M) [application/octet-stream]
Saving to: ‘pipeline.pickle’

pipeline.pickle     100%[===================>]   2.22M  --.-KB/s    in 0.08s   

2021-04-17 22:07:22 (29.4 MB/s) - ‘pipeline.pickle’

In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import pickle

wczytanie plików

In [4]:
with open('pipeline.pickle', 'rb') as handle:
    pipeline = pickle.load(handle)

pipeline

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categories='auto',
                                                                drop='first',
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['season',
                                                   'part_of_the_day'])],
                                   verbose=False)),
                ('model',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
 

10.1 Utworzenie siatki punktów w na terenie całego krakowa zlokalizowanych co 500m

In [14]:
points = pd.read_csv("https://raw.githubusercontent.com/ciepielajan/SDA_SMOG_Wawelski/main/Jan/model/points.csv")
points

,Unnamed: 0,id,latitude,longitude
0,0,1-1,50.10700,19.82000
1,1,1-2,50.10403,19.82000
2,2,1-3,50.10106,19.82000
3,3,1-4,50.09809,19.82000
4,4,1-5,50.09512,19.82000
...,...,...,...,...
1570,1570,45-31,50.01790,20.02372
1571,1571,45-32,50.01493,20.02372
1572,1572,45-33,50.01196,20.02372
1573,1573,45-34,50.00899,20.02372


10.2 Obliczenie predykcji dla wszystkich stacji pomiarowych dla konkretnej daty

In [24]:
def parse_season(x):
    if x in [4,5,6,7,8,9]:
       return "lato"
    elif x in [1,2,3,10,11,12]:
       return "zima"
    else:
       return "bład"

def parse_hours(x):
    if x in [10,11,12,13,14,15,16]:
       return "dzien"
    elif x in [17,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,9,]:
       return "noc"
    else:
       return "bład"

In [25]:
user_time = pd.Timestamp(data + " " + time + ":00:00")

In [26]:
user_date = pd.DataFrame([user_latitude, user_longitude]).T
user_date.columns = ["latitude","longitude"]
user_date["season"] = parse_season(user_time.month)
user_date["part_of_the_day"] = parse_hours(user_time.hour)
user_date

,latitude,longitude,season,part_of_the_day
0,50.057747,19.961374,zima,dzien


`do dodatkowych punktów dodaje dane po transformacji`

In [27]:
points_prediction = points[["latitude","longitude"]]
points_prediction["season"] = user_date["season"].values[0]
points_prediction["part_of_the_day"] = user_date["part_of_the_day"].values[0]
points_prediction[:3]

,latitude,longitude,season,part_of_the_day
0,50.10700,19.82,zima,dzien
1,50.10403,19.82,zima,dzien
2,50.10106,19.82,zima,dzien


In [28]:
#predykcja
predict = pipeline.predict(points_prediction)
predict

array([52.19840258, 52.19840258, 51.17661215, ..., 45.9599571 ,
       45.9599571 , 45.9599571 ])

In [29]:
points_prediction["prediction_pm10"] = predict
points_prediction[:3]

,latitude,longitude,season,part_of_the_day,prediction_pm10
0,50.10700,19.82,zima,dzien,52.198403
1,50.10403,19.82,zima,dzien,52.198403
2,50.10106,19.82,zima,dzien,51.176612


In [30]:
points_prediction["prediction_pm10"].describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
prediction_pm10,1575.0,53.641153,8.753608,37.92413,47.375972,52.271497,57.722245,91.716763


In [31]:
!pip -q install folium
import folium

In [32]:
location = [50.0614 ,19.9366]

m = folium.Map(location=location, zoom_start=17)


for ind in points_prediction.index: 
    tooltip_txt = 'PM10:' + str(points_prediction['prediction_pm10'][ind])
    
    if points_prediction['prediction_pm10'][ind] <= 25:
      color="green",
    elif (points_prediction['prediction_pm10'][ind] > 25) & (points_prediction['prediction_pm10'][ind] <= 50):
      color="orange",
    elif (points_prediction['prediction_pm10'][ind] > 50) & (points_prediction['prediction_pm10'][ind] <= 100):
      color="red",
    elif (points_prediction['prediction_pm10'][ind] > 100):
      color="black",
    else:
      color="white"

    folium.Circle(
          location=[float(points_prediction['latitude'][ind]), float(points_prediction['longitude'][ind])], 
          radius=165,
          # popup="Laurelhurst Park",
          # fill=True,
          # fill_color="#3186cc",
          fill_color=color,
          color=color,
          tooltip= tooltip_txt
      ).add_to(m)
m

In [34]:
name_file = "index.html"

m.save(name_file)
from google.colab import files
files.download(name_file) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>